In [2]:
%pip install transformers
%pip install evaluate
%pip install datasets
%pip install pandas
%pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux

# webkb-parsed.csv

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

path = 'webkb-parsed.csv'
atributoDeLabel = 'class'
atributoDeTexto = 'text'

# Carrega o dataset
data = pd.read_csv(path)

# Transforma os labels em números
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(data[atributoDeLabel].tolist())

data[atributoDeLabel] = labelencoder.transform(data[atributoDeLabel].tolist())

# Separa o dataset em treino, validação e teste
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data[atributoDeTexto], data[atributoDeLabel], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=2/3, random_state=42)

print(f"Tamanho do treino: {len(train_texts)}, validação: {len(val_texts)}, teste: {len(test_texts)}")

Tamanho do treino: 5797, validação: 828, teste: 1657


In [4]:
# Tokenizar textos
from transformers import AutoTokenizer

model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
# Montar modelo
from transformers import AutoModelForSequenceClassification

num_labels = len(data[atributoDeLabel].unique())

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# gerar datasets
train_dataset = Dataset.from_dict({ 'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': train_labels})
val_dataset = Dataset.from_dict({ 'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask'], 'labels': val_labels})
test_dataset = Dataset.from_dict({ 'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'labels': test_labels})

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-7-31dc9b9da707>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
trainer.train()
trainer.save_model('model_1')

Epoch,Training Loss,Validation Loss
1,0.795300,0.409227


In [9]:
prediction = trainer.predict(test_dataset)

In [10]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')
    conf_matrix = confusion_matrix(labels, preds)

    print(f"Acurácia: {acc:.4f}")
    print(f"F1-Score Micro: {f1_micro:.4f}")
    print(f"F1-Score Macro: {f1_macro:.4f}")
    print("Matriz de Confusão:\n", conf_matrix)

print("métricas do webkb-parsed")
compute_metrics(prediction)

métricas do webkb-parsed
Acurácia: 0.8618
F1-Score Micro: 0.8618
F1-Score Macro: 0.7166
Matriz de Confusão:
 [[169   1   2  23   0   0   0]
 [  1  28   0   3   2   0   1]
 [  0   0 195   4   2   0  31]
 [ 13   6   4 652  25   0  30]
 [  2   0   0  27  72   0   1]
 [  0   0   6   1   1   0  20]
 [  0   0   8  14   1   0 312]]


# SyskillWebert.csv

In [11]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

path = 'webkb-parsed.csv'
atributoDeLabel = 'class'
atributoDeTexto = 'text'

# Carrega o dataset
data = pd.read_csv(path)

# Transforma os labels em números
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(data[atributoDeLabel].tolist())

data[atributoDeLabel] = labelencoder.transform(data[atributoDeLabel].tolist())

# Separa o dataset em treino, validação e teste
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data[atributoDeTexto], data[atributoDeLabel], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=2/3, random_state=42)

print(f"Tamanho do treino: {len(train_texts)}, validação: {len(val_texts)}, teste: {len(test_texts)}")

Tamanho do treino: 5797, validação: 828, teste: 1657


In [12]:
# Tokenizar textos
from transformers import AutoTokenizer

model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)


In [13]:
# Montar modelo
from transformers import AutoModelForSequenceClassification

num_labels = len(data[atributoDeLabel].unique())

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# gerar datasets
train_dataset = Dataset.from_dict({ 'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': train_labels})
val_dataset = Dataset.from_dict({ 'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask'], 'labels': val_labels})
test_dataset = Dataset.from_dict({ 'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'labels': test_labels})

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-15-31dc9b9da707>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()
trainer.save_model('model_2')

Epoch,Training Loss,Validation Loss
1,0.941100,0.413693


In [17]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')
    conf_matrix = confusion_matrix(labels, preds)

    print(f"Acurácia: {acc:.4f}")
    print(f"F1-Score Micro: {f1_micro:.4f}")
    print(f"F1-Score Macro: {f1_macro:.4f}")
    print("Matriz de Confusão:\n", conf_matrix)

print('Métricas do SyskillWebert')
compute_metrics(trainer.predict(val_dataset))

Métricas do SyskillWebert


Acurácia: 0.8647
F1-Score Micro: 0.8647
F1-Score Macro: 0.7270
Matriz de Confusão:
 [[ 86   0   0  10   0   0   0]
 [  0  19   0   1   0   0   0]
 [  0   0  97   1   0   0  19]
 [ 16   2   3 362   8   0  15]
 [  0   0   1  15  29   0   0]
 [  0   0   4   0   1   0   7]
 [  0   0   4   5   0   0 123]]
